___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

###### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



## 1. Підключитися до API.


In [14]:
import requests
r = requests.get ('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date=20210817&json')
if r.status_code == 200:
    print ("Подключение к API произошло успешно")

Подключение к API произошло успешно


## 2. Отримати курс долара за останній рік.

In [36]:
import requests
r = requests.get ('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date=20210817&json')
if r.status_code == 200:
    data = r.json()
    for currency in data:
        if currency ['cc'] == 'USD':
            print (f'Курс доллара: {currency["rate"]}')

Курс доллара: 26.6457


In [115]:
import datetime

def currency_data(start_date, end_date):
    
    data_store = {}
    day = start_date

    while day != end_date:
        url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date='+day.strftime('%Y%m%d')+'&json?'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for currency in data:
                if currency["cc"] == "USD":
                    curr_name = currency["txt"]
                    rate = currency["rate"]
                    date = currency["exchangedate"]
                    data_store[day] = rate

        day = day + datetime.timedelta(days=1)
    
    return data_store
    
start_day = datetime.date(year = 2021, month = 1, day = 1)

import pandas as pd
data = {'Курс долара за останній рік': [currency_data(start_day, datetime.date.today())],
        }
df = pd.DataFrame (data)

print (df)







                         Курс долара за останній рік
0  {2021-01-01: 28.2746, 2021-01-02: 28.2746, 202...


## 3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [99]:
month_list = []
for i in range (7):
    start_date = datetime.date(year = 2021, month = i+1, day = 1)
    end_date = datetime.date(year = 2021, month = i+2, day = 1)
    
    month_list.append(currency_data(start_date, end_date))
    
import math

month_standard_deviation = {}
month_mean_value = {}
i = 1

for month in month_list:
    month_mean_value[f'month {i}'] = sum(month.values())/len(month.values())
    month_standard_deviation[f'month {i}'] = math.sqrt(sum(list(map(lambda x: (x - month_mean_value[f'month {i}'])**2, month.values())))/len(month.values()))
    i +=1
    
print("Mean values per month: ", month_mean_value, "\n")
print("Standard_deviation per month: ", month_standard_deviation)

Mean values per month:  {'month 1': 28.219770967741937, 'month 2': 27.88466071428571, 'month 3': 27.79555806451613, 'month 4': 27.930006666666667, 'month 5': 27.602848387096767, 'month 6': 27.23987333333332, 'month 7': 27.214548387096777} 

Standard_deviation per month:  {'month 1': 0.12115058663084945, 'month 2': 0.10439519125243586, 'month 3': 0.113636249822287, 'month 4': 0.08930549790217604, 'month 5': 0.11849255828264615, 'month 6': 0.1534613869639165, 'month 7': 0.15329376367376968}


## 4. Дану інформацію записати у файл за допомогою pickle.

In [100]:
import pickle

f = open('currency_data.pickle', 'wb')
data = {"Mean values per month":month_mean_value, "Standard_deviation per month" : month_standard_deviation}
pickle.dump(data, f)
f.close()

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [81]:
#pip install lxml
#pip install beautifulsoup4

In [87]:
import requests
import lxml
from bs4 import BeautifulSoup
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

for page in range(8):
        SITE = f'https://smallpacking.agrosem.ua/shop/page/{i}/'
        response = requests.get(SITE)
        soup = BeautifulSoup(response.text, 'lxml')
        product_cont = soup.find_all("div", class_ = "product-from-category-container")
        

link = product.a.get('href')

def get_price(link):
    r = requests.get(link, HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')
    p = soup.find('span', class_="regular-price").text
    q = len(p)
    price = int(''.join(p[0:len(p)-4:1].split()[0:2]))
    return price


def get_sku(link):
    r = requests.get(link, HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')
    i = soup.find('span', class_="sku").text
    sku = int(i[5:len(i)])
    return sku

def get_package(link):
    r = requests.get(link, HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')
    package = soup.find('span', class_="type-packaging-name").text
    return package


import sqlite3

connection = sqlite3.connect('agro_product_list.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS agro_product_list (name TEXT, SKU INT, pack_type TEXT, Price INT)''')

connection.commit()

for i in range(1, 9):
    SITE = f'https://smallpacking.agrosem.ua/shop/page/{i}/'
    r = requests.get(SITE, HEADERS)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        product = soup.find('div', class_="product-from-category-container")
        for k in product.parent.find_all('div', class_="product-from-category-container"):
            name = k.span.text
            link = k.a.get('href')
            sku = get_sku(link)
            price = get_price(link)
            package = get_package(link)
            cursor.execute("INSERT INTO agro_product_list VALUES (?,?,?,?)", (name, sku, package, price))
            print(name, package, sku, price)


Ferticare Hydro 6-14-30 + micro Коробка 28164 1900
Ferticare Kombi 14-11-25 + micro Коробка 66353 1700
MagPlon K (нітрат калію) Коробка 39566 1200
Pulan 34.4 Коробка 69156 7500
Saletrosan®26 Коробка 74266 4400
Saletrosan®30 Коробка 26252 6000
Saletrzak 27 macro Коробка 67325 4300
Siarczan AS 21 Macro Коробка 76913 4700
Yara Liva Кальциніт Коробка 51667 550
Yara Liva Нітрабор Коробка 95975 550
YaraBela Сульфан Коробка 93119 9500
YaraIrix Коробка 35951 23000
YaraMila Complex 12-11-18 Коробка 21521 850
YaraMila Cropcare 11-11-21 Коробка 31357 850
YaraMila NP 20-20 +9,0SO3 Коробка 47993 12600
YaraMila NPK 12-24-12 Коробка 89147 14700
YaraMila NPK 16-16-16 Коробка 84721 12200
YaraMila NPK 16-27-7 Коробка 63243 15600
YaraMila NPK 18-11-13 Коробка 24963 11300
YaraMila NPK 7-20-28 Коробка 24344 15600
YaraMila NPK 8-24-24 Коробка 64776 15800
YaraMila NPK 9-12-25 + 2MgO + 6,5SO3 + micro Коробка 66872 12800
YaraTera SOP (сульфат калію) Коробка 19571 1100
YaraVita Rexolin ABC (комплекс мікроелемен

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [51]:
import sqlite3
conn = sqlite3.connect (r'C:\Users\Magnolia\Desktop\Main Academy\SQLite\tysql.sqlite')
cursor = conn.cursor()
cursor.execute('SELECT sqlite_version()')
vers_tysql = cursor.fetchone()
print(f'Версія бази даних SQLite: {vers_tysql}')

cursor.execute("SELECT count(*) from sqlite_master as tables WHERE type='table'")
q_tables = cursor.fetchone()
print(f'База складається з {q_tables} таблиць')



Версія бази даних SQLite: ('3.36.0',)
База налічує (5,) таблиць
Складається з таких загловків <built-in method fetchall of sqlite3.Cursor object at 0x000001C32BF69C00>


2. Виведіть перелік всіх таблиць.

In [52]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
name_tables = cursor.fetchall()
print(f'Перелік назв таблиць: {name_tables}')

Перелік назв таблиць: [('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]


3. Список всіх cust_id з таблиці Customers table.

In [53]:
cursor.execute('SELECT cust_id FROM Customers')
cust_id = cursor.fetchall()
print(f'Список cust_id із таблиці Customers: {cust_id}')

Список cust_id із таблиці Customers: [('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]


4. Всю таблицю Customers table

In [57]:
cursor.execute('SELECT * FROM Customers')
all_cust = cursor.fetchall()
print(f'Таблиця Customers table:')
for i in all_cust:
    print(i)

Таблиця Customers table:
('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)


5. Список клієнтів (cust_names) відсортованих від Z до A

In [58]:
cursor.execute('SELECT cust_name FROM Customers ORDER BY cust_name DESC')
custo_names = cursor.fetchall()
print('Список клієнтів (cust_names), відсортованих від Z до A:')
for i in custo_names:
    print(i)

Список клієнтів (cust_names), відсортованих від Z до A:
('Village Toys',)
('The Toy Store',)
('Kids Place',)
('Fun4All',)
('Fun4All',)


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [62]:
cursor.execute("""SELECT cust_name, order_num, order_date
                FROM Orders 
                LEFT JOIN Customers 
                WHERE Orders.cust_id == Customers.cust_id
                ORDER BY cust_name, order_date""")
client_order = cursor.fetchall()
print(f'Таблиця клієнтів та замовлень, яка відсортована по клієнту та по даті замовлення: клієнт, номер, дата замовлення')
for i in client_order:
    print(i)

Таблиця клієнтів та замовлень, яка відсортована по клієнту та по даті замовлення: клієнт, номер, дата замовлення
('Fun4All', 20006, '2012-01-12')
('Fun4All', 20007, '2012-01-30')
('The Toy Store', 20008, '2012-02-03')
('Village Toys', 20009, '2012-02-08')
('Village Toys', 20005, '2012-05-01')


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [65]:
cursor.execute("""SELECT quantity, item_price
                FROM OrderItems 
                ORDER BY quantity DESC, item_price DESC""")
quan_price = cursor.fetchall()
print(f'Таблиця з кількістю та вартістю товару, відсортована в порядку спадання по кількості та вартості:')
for i in quan_price:
    print(i)

Таблиця з кількістю та вартістю товару, відсортована в порядку спадання по кількості та вартості:
(250, 2.49)
(250, 2.49)
(250, 2.49)
(100, 10.99)
(100, 5.49)
(100, 2.99)
(100, 2.99)
(100, 2.99)
(50, 11.49)
(50, 4.49)
(20, 5.99)
(10, 11.99)
(10, 8.99)
(10, 3.49)
(10, 3.49)
(10, 3.49)
(5, 11.99)
(5, 4.99)


8. Товар (з таблиці Products), ціна якого становить 9.49

In [66]:
cursor.execute('SELECT prod_name, prod_price FROM Products WHERE prod_price == 9.49')
product_name = cursor.fetchall()
print(f'Товари, ціна яких становить 9.49:')
for i in product_name:
    print(i)

Товари, ціна яких становить 9.49:
('King doll', 9.49)
('Queen doll', 9.49)


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [67]:
cursor.execute("""SELECT prod_name, prod_price FROM Products 
                WHERE prod_price BETWEEN 3 AND 6
                ORDER BY prod_price""")
name_price_3_6 = cursor.fetchall()
print(f'Назва і ціна товарів, які коштують в діапазоні від 3 до 6 та відсортовані по ціні в порядку зростання')
for i in name_price_3_6:
    print(i)

Назва і ціна товарів, які коштують в діапазоні від 3 до 6 та відсортовані по ціні в порядку зростання
('Fish bean bag toy', 3.49)
('Bird bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)
('8 inch teddy bear', 5.99)


10. Кількість товару, що було продано

In [68]:
cursor.execute("SELECT sum(quantity) FROM OrderItems")
sold_quan = cursor.fetchone()
print(f'Всього було продано таку кількість товарів: {sold_quan}')

Всього було продано таку кількість товарів: (1430,)


11. Кількість найменувань товару, ціна якого більша за 4

In [69]:
cursor.execute("SELECT count(prod_name) FROM Products WHERE prod_price > 4")
prod_name_mt4 = cursor.fetchone()
print(f'Кількіть найменувань товару, ціна якого більша за 4: {prod_name_mt4}')

Кількіть найменувань товару, ціна якого більша за 4: (6,)


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [73]:
cursor.execute("SELECT prod_name, prod_price FROM Products ORDER BY prod_price DESC")
prod_name_w3 = cursor.fetchmany(3)
print(f'Три найдорожчих товара в базі:')
for i in prod_name_w3:
    print(i)

Три найдорожчих товара в базі:
('18 inch teddy bear', 11.99)
('King doll', 9.49)
('Queen doll', 9.49)


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [75]:
cursor.execute("""SELECT cust_name, count(order_num)
                FROM Orders
                LEFT JOIN Customers 
                WHERE Orders.cust_id == Customers.cust_id
                GROUP BY cust_name""")
ord_cust_count = cursor.fetchall()
print(f'Кількість замовлень для кожного клієнта:')
for i in ord_cust_count:
    print(i)

Кількість замовлень для кожного клієнта:
('Fun4All', 2)
('The Toy Store', 1)
('Village Toys', 2)


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [76]:
cursor.execute("""SELECT cust_name, order_num 
                FROM Customers JOIN Orders ON Customers.cust_id == Orders.cust_id
                ORDER BY cust_name""")
cust_orders = cursor.fetchall()
print(f'Клієнти та їх замовлення:')
for i in cust_orders:
    print(i)

Клієнти та їх замовлення:
('Fun4All', 20006)
('Fun4All', 20007)
('The Toy Store', 20008)
('Village Toys', 20005)
('Village Toys', 20009)


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [79]:
cursor.execute ("SELECT prod_name FROM Products WHERE prod_price < 5")
prod_name5 = cursor.fetchall()
print(f'Товари,ціна яких менша 5:')
for i in prod_name5:
    print(i)

cursor.execute("SELECT prod_name FROM Products WHERE prod_price >= 5")
prod_name_m5 = cursor.fetchall()
print(f'Товари, ціна яких більша рівна 5:')
for i in prod_name_m5:
    print(i)

cursor.execute("""SELECT prod_name FROM Products WHERE prod_price < 5
            UNION ALL
            SELECT prod_name FROM Products WHERE prod_price >= 5""")
union_2 = cursor.fetchall()
print(f"Об'єднана таблиця:")
for i in union_2:
    print(i)
    
conn.commit()
conn.close()

Товари,ціна яких менша 5:
('Fish bean bag toy',)
('Bird bean bag toy',)
('Rabbit bean bag toy',)
('Raggedy Ann',)
Товари, ціна яких більша рівна 5:
('8 inch teddy bear',)
('12 inch teddy bear',)
('18 inch teddy bear',)
('King doll',)
('Queen doll',)
Об'єднана таблиця:
('Fish bean bag toy',)
('Bird bean bag toy',)
('Rabbit bean bag toy',)
('Raggedy Ann',)
('8 inch teddy bear',)
('12 inch teddy bear',)
('18 inch teddy bear',)
('King doll',)
('Queen doll',)


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


*1.Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd*

SELECT model, speed, hd FROM PC WHERE price < 500

*2. Найдите производителей принтеров. Вывести: maker*

SELECT DISTINCT maker FROM Product WHERE type = 'Printer'

*3. Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.*

SELECT model, ram, screen FROM Laptop WHERE price > 1000

*4.Найдите все записи таблицы Printer для цветных принтеров.*

SELECT * FROM Printer WHERE color = 'y'

*5. Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол. *

SELECT model, speed, hd FROM PC WHERE price < 600 AND (cd = '12x' OR cd = '24x')

*6. Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость. *

SELECT DISTINCT Product.maker, Laptop.speed FROM Laptop JOIN Product 
ON Product.model = Laptop.model WHERE hd >= 10 ORDER BY maker, speed

*7. Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква). *

SELECT * FROM (SELECT model, price FROM PC UNION SELECT model, price FROM Laptop UNION SELECT model, price FROM Printer) AS a WHERE model IN (SELECT model FROM Product WHERE maker = 'B')


SELECT model FROM Product WHERE maker = 'B'

*8. Найдите производителя, выпускающего ПК, но не ПК-блокноты. *

SELECT DISTINCT maker FROM Product WHERE type = 'PC' EXCEPT SELECT maker FROM Product WHERE type = 'Laptop'

*9. Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker *

SELECT DISTINCT maker as Maker FROM Product JOIN PC ON Product.model = PC.model WHERE speed >= 450

*10. Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price *

SELECT model, price FROM Printer WHERE price=(SELECT MAX(price) FROM Printer)

*11. Найдите среднюю скорость ПК. *

SELECT AVG(speed) FROM PC

*12. Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол. *

SELECT AVG (speed) FROM Laptop WHERE price > 1000

*13. Найдите среднюю скорость ПК, выпущенных производителем A. *

SELECT AVG(speed) FROM PC INNER JOIN Product on Product.model = PC.model WHERE product.maker = 'A'

*14. Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий. *

SELECT Classes.class, Ships.name, country FROM Classes JOIN Ships ON Classes.class = Ships.class WHERE numGuns >= 10

*15. Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD *

SELECT hd FROM PC GROUP BY hd HAVING COUNT(model) >=2

*16. Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM. *

SELECT DISTINCT one.model, sec.model, one.speed, one.ram FROM PC one JOIN (SELECT speed, ram FROM PC GROUP BY speed, ram HAVING COUNT(speed) >= 2 AND COUNT(ram) >= 2) S ON one.speed = S.speed AND one.ram = S.ram JOIN PC sec ON sec.speed = S.speed AND sec.ram = S.ram AND sec.model < one.model ORDER BY one.model, sec.model

*17. Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК. Вывести: type, model, speed *

SELECT DISTINCT Product.type, Laptop.model, Laptop.speed FROM PC, Laptop, Product WHERE type = 'Laptop' AND Laptop.speed < ALL (SELECT PC.speed FROM PC)

*18. Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price *

SELECT maker, price FROM Product JOIN Printer ON Product.model = Printer.model WHERE color = 'y' AND price IN (SELECT MIN(price) FROM Printer WHERE color = 'y') GROUP BY maker, price

*19. Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов. Вывести: maker, средний размер экрана. *


SELECT maker, AVG(screen) as Avg_screen FROM Product JOIN Laptop ON Product.model = Laptop.model GROUP BY maker

*20. Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК. *

SELECT maker, COUNT(type) AS Count_Model FROM Product WHERE type = 'PC' GROUP BY maker HAVING COUNT(type) >= 3

*21. Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC. Вывести: maker, максимальная цена. *

SELECT maker, MAX(price) as Max_price FROM Product JOIN PC ON Product.model = PC.model GROUP BY maker

*22. Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена. *

SELECT speed, AVG(price) as Avg_price FROM PC WHERE speed > 600 GROUP BY speed

*23. Найдите производителей, которые производили бы как ПК со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц. Вывести: Maker*

SELECT maker FROM Product JOIN PC ON Product.model = PC.model WHERE type = 'PC' AND speed >= 750 INTERSECT SELECT maker FROM Product JOIN Laptop ON Product.model = Laptop.model WHERE type = 'Laptop' AND speed >= 750

*24. Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции. *

SELECT DISTINCT model FROM (SELECT model, price FROM PC UNION ALL SELECT model, price FROM Laptop UNION ALL SELECT model, price FROM Printer ) DB WHERE price = (SELECT MAX(price) FROM (SELECT model, price FROM PC UNION ALL SELECT model, price FROM Laptop UNION ALL SELECT model, price FROM Printer ) NT)

*25. Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker *

SELECT maker FROM Product WHERE type = 'printer' AND maker IN (SELECT maker FROM Product JOIN PC ON Product.model = PC.model WHERE ram = (SELECT MIN(ram) FROM PC) AND speed = (SELECT MAX(speed) FROM PC WHERE ram = (SELECT MIN(ram) FROM PC))) GROUP BY maker